In [2]:
import sys
!pip install smartsheet-python-sdk

You are using pip version 19.0.1, however version 19.0.3 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [11]:
import smartsheet
import logging
import os.path

In [12]:
# Initialize client
newsheet = smartsheet.Smartsheet('eaa1r3rmg8qibumjcvsgjsku82')

# Make sure we don't miss any errors
newsheet.errors_as_exceptions(True)

In [13]:
# Log all calls
logging.basicConfig(filename='rwsheet.log', level=logging.INFO)

sheetID = 3610680340309892



# Load entire sheet
sheet = newsheet.Sheets.get_sheet(sheetID)

print ("Loaded " + str(len(sheet.rows)) + " rows from sheet: " + sheet.name)

Loaded 2 rows from sheet: Testing


In [15]:
print(sheet)


{"accessLevel": "OWNER", "columns": [{"id": 5852237885925252, "index": 0, "primary": true, "title": "SiteID", "type": "TEXT_NUMBER", "validation": false, "version": 0, "width": 150}, {"id": 3600438072240004, "index": 1, "title": "City", "type": "TEXT_NUMBER", "validation": false, "version": 0, "width": 150}, {"id": 8104037699610500, "index": 2, "title": "State", "type": "TEXT_NUMBER", "validation": false, "version": 0, "width": 150}, {"id": 785688305133444, "index": 3, "title": "Zip", "type": "TEXT_NUMBER", "validation": false, "version": 0, "width": 150}, {"id": 5289287932503940, "index": 4, "title": "Street Address", "type": "TEXT_NUMBER", "validation": false, "version": 0, "width": 150}, {"id": 3037488118818692, "index": 5, "title": "Action", "type": "TEXT_NUMBER", "validation": false, "version": 0, "width": 150}], "createdAt": "2019-04-09T18:44:39+00:00Z", "dependenciesEnabled": false, "effectiveAttachmentOptions": ["EVERNOTE", "ONEDRIVE", "BOX_COM", "DROPBOX", "EGNYTE", "FILE", "G

In [19]:
from smartsheet import *
import pandas as pd
#from credentials import smartsheet_token

smartsheet = smartsheet.Smartsheet('eaa1r3rmg8qibumjcvsgjsku82')

def get_sheet_as_df(sheet_id):
    ss1 = smartsheet.Sheets.get_sheet(sheet_id, page_size=0)
    row_count = ss1.total_row_count
    ss1 = smartsheet.Sheets.get_sheet(sheet_id, page_size=row_count)
    df = get_values(ss1)
    s2 = get_columns(ss1)
    df.columns = s2
    return df

def get_columns(ss):
    cl = ss.get_columns()
    d3 = cl.to_dict()
    df = pd.DataFrame(d3['data'])
    df = df.set_index('id')
    return df.title

def get_values(ss):
    d = ss.to_dict()
    drows = d['rows']
    rownumber = [x['rowNumber'] for x in drows]
    rows = [x['cells'] for x in drows]
    values = [[x['displayValue'] for x in y] for y in rows]
    return pd.DataFrame(values)

def check_overlaps(list1, list2):
#check for overlaps and remainders
#from http://stackoverflow.com/questions/5094083/find-the-overlap-between-2-python-lists
    intersection = set(list1) & set(list2)
    l1_remainder = set(list1) - set(list2)
    l2_remainder = set(list2) - set(list1)
    d = {'intersection':intersection,
        'list1_remainder': l1_remainder,
        'list2_remainder': l2_remainder
        }
    return d

In [22]:
sheetdf = get_sheet_as_df(3610680340309892)
sheetdf

title,SiteID,City,State,Zip,Street Address,Action
0,Site123,Toronto,Ontario,Y5p5K9,The Street,This street doesn't exist
1,Site456,NewYork,NewYork,32523,The Other Street,This street also doesn't exist


In [29]:
sheetdf.SiteID[1]

'Site456'

In [30]:
!pip install pdfrw

You are using pip version 19.0.1, however version 19.0.3 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [33]:
#checking how to map dataframe values to dictionary

data_dict = {
   'business_name_1': sheetdf.SiteID[1]
}
data_dict

{'business_name_1': 'Site456'}

In [31]:
#not working yet, need to specify template file 
#and map the pdf fillable fields with correct dataframe fields

import os
import pdfrw


INVOICE_TEMPLATE_PATH = 'invoice_template.pdf'
INVOICE_OUTPUT_PATH = 'invoice.pdf'


ANNOT_KEY = '/Annots'
ANNOT_FIELD_KEY = '/T'
ANNOT_VAL_KEY = '/V'
ANNOT_RECT_KEY = '/Rect'
SUBTYPE_KEY = '/Subtype'
WIDGET_SUBTYPE_KEY = '/Widget'


def write_fillable_pdf(input_pdf_path, output_pdf_path, data_dict):
    template_pdf = pdfrw.PdfReader(input_pdf_path)
    annotations = template_pdf.pages[0][ANNOT_KEY]
    for annotation in annotations:
        if annotation[SUBTYPE_KEY] == WIDGET_SUBTYPE_KEY:
            if annotation[ANNOT_FIELD_KEY]:
                key = annotation[ANNOT_FIELD_KEY][1:-1]
                if key in data_dict.keys():
                    annotation.update(
                        pdfrw.PdfDict(V='{}'.format(data_dict[key]))
                    )
    pdfrw.PdfWriter().write(output_pdf_path, template_pdf)


data_dict = {
   'business_name_1': 'Bostata',
   'customer_name': 'company.io',
   'customer_email': 'joe@company.io',
   'invoice_number': '102394',
   'send_date': '2018-02-13',
   'due_date': '2018-03-13',
   'note_contents': 'Thank you for your business, Joe',
   'item_1': 'Data consulting services',
   'item_1_quantity': '10 hours',
   'item_1_price': '$200/hr',
   'item_1_amount': '$2000',
   'subtotal': '$2000',
   'tax': '0',
   'discounts': '0',
   'total': '$2000',
   'business_name_2': 'Bostata LLC',
   'business_email_address': 'hi@bostata.com',
   'business_phone_number': '(617) 930-4294'
}

if __name__ == '__main__':
    write_fillable_pdf(INVOICE_TEMPLATE_PATH, INVOICE_OUTPUT_PATH, data_dict)

PdfParseError: Could not read PDF file invoice_template.pdf